<a href="https://colab.research.google.com/github/katie0107/ECON211_decision_tree_katie/blob/main/DecisionTree_Katie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Data Wrangling

In [ ]:
import pandas as pd 
import numpy as np
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics 
clf = DecisionTreeClassifier()
import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
###show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df=pd.read_csv('https://raw.githubusercontent.com/katie0107/ECON211_decision_tree_katie/main/insurance.csv')
#df_ESG= df_ESG[df_ESG.Month==1]
df.head()

In [ ]:
df = df.dropna()

In [ ]:
df['smoker_01']=df['smoker'].apply(lambda x: 1 if x=='yes' else 0)
df['sex_01']=df['sex'].apply(lambda x: 1 if x=='female' else 0)
df.head()

In [ ]:
df['charges_level']=pd.qcut(df.charges,5,labels=np.arange(1,6,1) )
df['charges_level_str']=df['charges_level'].apply(lambda x: 'Very High' if x==5 else 'High' if x==4 else 'Middle' if x==3 else 'Low' if x==2 else 'Very Low')
df.head()

In [ ]:
order = ['age', 'sex_01', 'bmi', 'children', 'smoker_01', 'charges']
df_new = df[order]

In [ ]:
import seaborn as sns 
# compute the pair wise correlation for all columns  
correlation_matrix = df_new.corr().round(2)

# print heatmap of the correlation matrix
# next two lines remove the duplicate upper half of matrix
mask = np.zeros(correlation_matrix.shape, dtype=bool)
mask[np.triu_indices(len(mask),1)] = True

sns.heatmap(data=correlation_matrix, annot=True, cmap = "coolwarm", mask = mask); 

As can be seen from the correlation map above, 'charges' is most relevant to 'smoker_01' and the second prominent feature is 'age'.

# Build Decision Tree

In [ ]:
feature_names=['smoker_01', 'age']
features=df[feature_names]
targets=df['charges_level_str']
targets_names = targets.unique()

In [ ]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_targets, test_targets = train_test_split(features, targets, test_size=0.2, random_state=42)

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [ ]:
#####Optimizing Decision Tree Performance
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(criterion='entropy', max_depth=3,random_state=0)

# Train Decision Tree Classifer
clf = clf.fit(train_features,train_targets)

#Predict the response for test dataset
y_pred = clf.predict(test_features)

# Compute test set accuracy  
acc = accuracy_score(y_pred, test_targets)
print("Test set accuracy: {:.2f}".format(acc))

In [ ]:
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus

In [ ]:
dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = feature_names,class_names=clf.classes_)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('ESG.png')
Image(graph.create_png())